In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# To install PIL, look for the Pillow package on pip
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import random as rand
from sklearn.decomposition import TruncatedSVD

In [3]:
# Draw a kindergarten worthy background complete with a token sun and a house
im = Image.new( "RGB", (800, 600), '#666666' ) # the sky
ImageDraw.Draw( im ).rectangle( (0, 500, 800, 600), "#888888" ) # the ground
ImageDraw.Draw( im ).rectangle( (100, 400, 250, 500), "#bbbbbb" ) # the house
ImageDraw.Draw( im ).rectangle( (150, 450, 200, 500), "#444444" ) # the door
ImageDraw.Draw( im ).rectangle( (110, 420, 140, 450), "#666666" ) # the left window
ImageDraw.Draw( im ).rectangle( (210, 420, 240, 450), "#666666" ) # the right window
ImageDraw.Draw( im ).polygon( (80, 400, 270, 400, 175, 350), "#555555" ) # the roof
ImageDraw.Draw( im ).ellipse( (650, 50, 750, 150), "#bbbbbb" ) # the sun

# Create animated foreground with a ball bouncing around and a flying piece of text
# Warning: the method does not handle slow moving elements very well since they confuse
# the method into thinking that slow moving elements are background elements. To see this,
# decrease the speed of either the ball or the text by reducing the absolute value of
# the increments to posx, posy, posr and you may find residual (static) ghosts in the background

# The above is not very surprising though -- if a person is standing (absolutely) still in a
# video, would we call that person a part of the background or the foreground?
frames = []
posx = 850
posy = 300
posr = 0
numFrames = 20
for n in range( numFrames ):
    frame = im.copy()
    draw = ImageDraw.Draw( frame )
    draw.ellipse( (posx, posy, posx + 50, posy + 50), "white" )
    draw.multiline_text( (posr, posr), "CS771, IITK", "white", ImageFont.truetype("ariblk.ttf", 50) )
    frames.append( frame )
    posx -= 25
    posy += 20
    posr += 15
for n in range( numFrames ):
    frame = im.copy()
    draw = ImageDraw.Draw( frame )
    draw.ellipse( (posx, posy, posx + 50, posy + 50), "white" )
    draw.multiline_text( (posr, posr), "CS771, IITK", "white", ImageFont.truetype("ariblk.ttf", 50) )
    frames.append( frame )
    posx -= 25
    posy -= 20
    posr += 15
for n in range( numFrames ):
    frame = im.copy()
    draw = ImageDraw.Draw( frame )
    draw.ellipse( (posx, posy, posx + 50, posy + 50), "white" )
    draw.multiline_text( (posr, posr), "CS771, IITK", "white", ImageFont.truetype("ariblk.ttf", 50) )
    frames.append( frame )
    posx += 25
    posy -= 20
    posr += 15
for n in range( numFrames ):
    frame = im.copy()
    draw = ImageDraw.Draw( frame )
    draw.ellipse( (posx, posy, posx + 50, posy + 50), "white" )
    draw.multiline_text( (posr, posr), "CS771, IITK", "white", ImageFont.truetype("ariblk.ttf", 50) )
    frames.append( frame )
    posx += 25
    posy += 20
    posr += 15
frames[0].save( "bounce_cs771.gif", save_all = True, append_images = frames[1:], duration = 40, loop = 0 )

In [4]:
# Handling full color images is a pain since we would have to do foreground/background
# extraction separately for all three channels i.e. R, G, B. Easier to convert the GIF
# to a grayscale format so that there is only one channel to worry about
def convertToGrayscale( filename, newFilename ):
    im = Image.open( filename )
    newFrames = []
    try:
        while True:
            newFrame = im.convert( mode = "L" )
            newFrames.append( newFrame )
            im.seek( im.tell() + 1 )
    except EOFError:
        pass
    newFrames[0].save( newFilename, save_all = True, append_images = newFrames[1:], duration = im.info["duration"], loop = 0 )
    # Dont forget to close files to prevent handle/memory leaks
    im.close()
    
# Find out how many frames are there in this GIF animation video
def getLength( filename ):
    im = Image.open( filename )
    numFrames = 0
    try:
        while True:
            numFrames += 1
            im.seek( im.tell() + 1 )
    except EOFError:
        pass
    # Dont forget to close files to prevent handle/memory leaks
    im.close()
    return numFrames

# Extract the frames from this GIF file and return all frames stacked in a matrix
def getData( filename ):
    numFrames = getLength( filename )
    im = Image.open( filename )
    data = np.zeros( (numFrames, np.prod( np.array( im.size ) )) )
    try:
        while True:
            frame = im.convert( mode = "L" )
            data[im.tell()] = np.array( list( frame.getdata() ) )
            im.seek( im.tell() + 1 )
    except EOFError:
        pass
    # Dont forget to close files to prevent handle/memory leaks
    im.close()
    return data, numFrames

In [5]:
# Retain only the top few entries in a matrix (by magnitude)
# and set everything else to zero
def applyHardThresholding( X, numPixels ):
    (n, d) = X.shape
    arr = X.reshape(-1)
    idx = np.argsort( np.abs( X.reshape(-1) ) )[::-1]
    XThresh = np.zeros( idx.shape )
    XThresh[idx[:numPixels * n]] = arr[idx[:numPixels * n]]
    return XThresh.reshape( (n,d) ), idx.reshape( (n,d) )

# Do "robust" PCA i.e. extract leading components even when the
# matrix may have corruptions (foreground elements)
# Warning: this is a highly simplified version of the algorithm
# For a more effective algorithm (also more involved), please see
# Netrapalli et al, Non-convex Robust PCA, NIPS 2014

# X: a matrix that we suspect is low rank but for some corruptions
# i.e. X = L + S where L is a low-rank matrix and S is a sparse matrix
# (sparse because there is not too much foreground in any frame)

# r: the rank we anticipate for L. For us r = 1 since background is static
# niterAltOpt: for how many iteration should we run alternating optimization?
# numFGPixels: a (rough) upper bound on the number of foreground pixels in any frame
def doRPCA( X, r = 1, niterAltOpt = 0, numFGPixels = 0 ):
    (n, d) = X.shape
    svd = TruncatedSVD( n_components = 1, n_iter = 10 )
    # Extract the leading right singular vector of X
    # If life were simple (for example if there had been no
    # foreground at all), this would have been the background image
    svd.fit( X )
    L = svd.inverse_transform( svd.transform( X ) )
    # Life is not so simple and the foreground elements will somewhat corrupt
    # the background so extract the foreground by looking at pixels that differ
    # the most from what we think is currently the background iamge
    S, idx = applyHardThresholding( X - L, numFGPixels )
    
    # Given an estimate of the foreground, do PCA to extract the background
    # Then given the background, extract the background by hard thresholding
    for t in range( niterAltOpt ):
        svd = TruncatedSVD( n_components = r, n_iter = 10 )
        clean = X - S
        # May also perform SVD on a random set of data points to improve speed
        # e.g. svd.fit( clean[np.random.choice(n, nSamples, replace = False)] )
        svd.fit( clean )
        L = svd.inverse_transform( svd.transform( clean ) )
        (S, idx) = applyHardThresholding( X - L, numFGPixels )
            
    return S, X - S

filename = "bounce_cs771.gif"
data, numFrames = getData( filename )
# Normalizing pixel values avoids very large values in computations
dataNorm = data/256
# Alternating optimization is actually beneficial
# Try setting niterAltOpt = 0 and you will find ghost images in the background
(foreground, background) = doRPCA( dataNorm, r = 1, niterAltOpt = 1, numFGPixels = 13000 )

In [6]:
# Un-normalize the matrix entries and make them integers
# so that they make sense as pixel values
def cleanup( X ):
    X = np.around( X * 256 )
    X[X < 0] = 0
    X[X > 255] = 255
    X = X.astype(int)
    return X

foreground = cleanup( foreground )
background = cleanup( background )

In [7]:
# Save the extracted foreground and background images as GIF files
# so that we can enjoy the animation

newFrames = []
for i in range( numFrames ):
    newFrame = Image.new( "L", im.size )
    newFrame.putdata( foreground[i] )
    newFrames.append( newFrame )
newFrames[0].save( 'foreground.gif', save_all = True, append_images = newFrames[1:], duration = 40, loop = 0 )

newFrames = []
for i in range( numFrames ):
    newFrame = Image.new( "L", im.size )
    newFrame.putdata( background[i] )
    newFrames.append( newFrame )
newFrames[0].save( 'background.gif', save_all = True, append_images = newFrames[1:], duration = 40, loop = 0 )